## Import

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Exploring data

### Read raw data from file

In [24]:
raw_df = pd.read_csv("../data/raw/data.csv")

In [25]:
raw_df.sample(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
3891,s3892,Movie,ReMastered: Devil at the Crossroads,Brian Oakes,NaN,United States,"April 26, 2019",2019,TV-14,49 min,"Documentaries, Music & Musicals","Cloaked in mystery, bluesman Robert Johnson le..."
7421,s7422,Movie,Mater,Pablo D'Alo Abba,"Lautaro Perotti, Tamara Kiper, Inda Lavalle, M...",Argentina,"August 1, 2018",2017,TV-MA,80 min,"Dramas, Independent Movies, International Movies",A single man in his 30s overwhelmed by his dom...
1137,s1138,Movie,The Knight and the Princess,Bashir El Deek,"Sedky Sakhr, Hashem El Garhy, Shahira Kamal, A...","Egypt, Saudi Arabia","April 1, 2021",2019,TV-PG,96 min,"Action & Adventure, International Movies, Musi...",A fictional account of the heroic quests of a ...
8343,s8344,Movie,The Heroes of Evil,Zoe Berriatúa,"Emilio Palacios, Jorge Clemente, Beatriz Medin...",Spain,"June 1, 2016",2015,TV-MA,99 min,"Dramas, International Movies","Three teenage misfits find camaraderie in sex,..."
5970,s5971,Movie,​Mayurakshi,Atanu Ghosh,"Soumitra Chatterjee, Prasenjit Chatterjee, Ind...",India,"September 15, 2018",2017,TV-14,100 min,"Dramas, International Movies",When a middle-aged divorcee returns to Kolkata...


### How many rows and how many columns does the raw data have?

In [27]:
shape = raw_df.shape
print(f"Data has {shape[0]} rows and {shape[1]} columns")

Data has 8807 rows and 12 columns


### What is the meaning of each row?

Each row represents information about a movie/ tv show on Netflix from 2008 to mid 2021.

### Are there duplicated rows?

In [29]:
unique_rows = raw_df.drop_duplicates().shape[0] - shape[0]
print(f'Number of duplicate rows: {unique_rows}')

Number of duplicate rows: 0


### What is the meaning of each column?

| Column Name | Description |
| --- | --- |
| show_id | Unique ID for each movie/ tv show |
| type | Type of the movie/ tv show (Movie or TV Show) |
| title | Title of the movie/ tv show |
| director | Director of the movie/ tv show |
| cast | Cast of the movie/ tv show |
| country | Country where the movie/ tv show was produced |
| date_added | Date when the movie/ tv show was added to Netflix |
| release_year | Year when the movie/ tv show was released |
| rating | Rating of the movie/ tv show |
| duration | Duration of the movie/ tv show |
| listed_in | Genre of the movie/ tv show |
| description | Description of the movie/ tv show |

### What is the current data type of each column? Are there columns having inappropriate data types?


In [30]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


### With each numerical column, how are values distributed?

### With each categorical column, how are values distributed?